# Imports

In [1]:
%%time 

from IPython.display import clear_output
from gc import collect
!pip install -q autogluon.tabular --force-reinstall;

collect();
clear_output();

CPU times: total: 46.9 ms
Wall time: 1.44 s


In [2]:
%%time 

from IPython.display import clear_output
from gc import collect

from warnings import filterwarnings
filterwarnings('ignore')

from IPython.display import display_html, clear_output
from pprint import pprint
from functools import partial
from copy import deepcopy
import pandas as pd, numpy as np, os, joblib
import re

from warnings import filterwarnings
filterwarnings('ignore')
from tqdm.notebook import tqdm

# AutoML package:-
from autogluon.tabular import TabularDataset, TabularPredictor;

print()
collect()


CPU times: total: 1.72 s
Wall time: 1.99 s


0

In [3]:
target   = "Rings";

train    = pd.read_csv("data/train.csv", index_col = "id")
test     = pd.read_csv("data/test.csv", index_col = "id")
sub_fl   = pd.read_csv("data/sample_submission.csv",)


train[target] = np.log1p(train[target].values);

print();
collect();

In [4]:
%%time 

model = TabularPredictor(label        = target,
                         problem_type = "regression",
                         eval_metric  = "root_mean_squared_error",
                         verbosity    = 2,
                         )

model.fit(train.drop(columns = ['id', "Source"], errors = "ignore"),
          save_space      = True,
          presets         = "optimize_for_deployment",
          time_limit      = 9000,
          excluded_model_types = ["NN_TORCH", "FASTAI", "NN"]
          )

preds = model.predict(test.drop(columns = ['id', "Source"], errors = "ignore"))

clear_output();

print()
display(model.leaderboard().\
        style.format(precision = 5).\
        set_caption(f"\nModel Leaderboard\n")
       )

collect()
print()

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-0.14529,root_mean_squared_error,0.11200,153.84165,0.00100,0.20003,2,True,4
1,LightGBM,-0.14565,root_mean_squared_error,0.02200,2.68181,0.02200,2.68181,1,True,1
2,CatBoost,-0.14661,root_mean_squared_error,0.00900,136.50481,0.00900,136.50481,1,True,3
3,RandomForestMSE,-0.14914,root_mean_squared_error,0.08000,14.45500,0.08000,14.45500,1,True,2



CPU times: total: 12min 7s
Wall time: 3min 11s


In [6]:
%%time 

sub_fl[target] = np.expm1(preds.values)

# Blending with good public work:-
sub1 = pd.read_csv(f"data/sample_submission.csv")[target].values

sub_fl[target] = np.average(np.c_[sub1, sub_fl[target].values], axis=1, weights = [0.80, 0.20])

sub_fl[target] = sub_fl[target].clip(1, 29);
sub_fl[target] = np.where(sub_fl[target].between(27.5,29), 29, sub_fl[target])
sub_fl.to_csv(f"outputs/submission_autogluon.csv", index = None)

CPU times: total: 141 ms
Wall time: 150 ms


public score `0.25593`